# Continuous Control

---

Congratulations for completing the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control an agent in a more challenging environment, where the goal is to train a creature with four arms to walk forward.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Crawler.app"`
- **Windows** (x86): `"path/to/Crawler_Windows_x86/Crawler.exe"`
- **Windows** (x86_64): `"path/to/Crawler_Windows_x86_64/Crawler.exe"`
- **Linux** (x86): `"path/to/Crawler_Linux/Crawler.x86"`
- **Linux** (x86_64): `"path/to/Crawler_Linux/Crawler.x86_64"`
- **Linux** (x86, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86"`
- **Linux** (x86_64, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86_64"`

For instance, if you are using a Mac, then you downloaded `Crawler.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Crawler.app")
```

In [ ]:
env = UnityEnvironment(file_name='app/Crawler.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
def randomSteps():
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
    env.close()

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
def ppo(env, agent, n_episodes=1700, max_t=2500, print_every=100, n_agent = 12, ini_epi=1):
    """
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        print_every (int): number of episodes to print result
        n_agent (int): number of identical agents in environment
    """
    brain_name = env.brain_names[0]
    scores_deque = deque(maxlen=print_every)
    t_mean = deque(maxlen=print_every//100)
    scores = []
    best_score = 0.
    for i_episode in range(ini_epi, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        agent_scores = np.zeros(n_agent)
        for t in range(max_t):
            actions, log_probs, _, values = agent.act(states)
            # get needed information from environment
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            rewards = np.where(np.isnan(rewards), 0, rewards)
            dones = np.array([1 if t else 0 for t in env_info.local_done])
            agent.save_step([states, values.detach(), actions, log_probs.detach(), rewards, 1 - dones])
            states = next_states
            agent_scores += rewards
            if np.all(dones) or (t+1) == max_t:
                t_mean.append(t+1)
            if (t+1) == max_t:
                break 
            
        score = np.mean(agent_scores)
        scores_deque.append(score)
        scores.append(score)

        if score > best_score:
            torch.save(agent.actor_critic.state_dict(), 'checkpoint.pth')
            best_score = score

        if i_episode % (print_every) == 0:
            print('\rEpisode {}\tCurrent Episode Average Score: {:.2f}\tAverage Score on 100 Episode: {:.2f}\tAverage Step: {:.0f}'.format(i_episode, 
                        score, np.mean(scores_deque), np.mean(t_mean)))

    return scores

In [ ]:
env = UnityEnvironment(file_name='app/Crawler_Linux_NoVis/Crawler.x86_64')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
# number of agents
n_agent = len(env_info.agents)
print("there are %d identical agents in the env"%n_agent)
# size of each action
action_size = brain.vector_action_space_size
# size of state space 
state_size = env_info.vector_observations.shape[1]
# train

In [ ]:
from utils.agent_crawler_ppo import Agent
from collections import deque
from datetime import datetime
import torch

    
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2, n_agent=n_agent)
agent.actor_critic.load_state_dict(torch.load('saved_models/checkpoint_Mult_crawler.pth', map_location=torch.device('cpu')))



In [ ]:
scores = ppo(env, agent, n_agent=n_agent, ini_epi=800)
# plot
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
fig.savefig('score.png')
plt.close(fig)
